In [1]:
import pandas as pd
import numpy as np
import re
import string
import jieba
import collections
import pickle
import codecs
import mysql.connector as sql

In [11]:
db_connection = sql.connect(host='rm-2zet5lw17as40fty28o.mysql.rds.aliyuncs.com',
                            database='medo_master', 
                            user='snowball', 
                            password='MEDOsnow$%^&')
# mesh_zh_df = pd.read_sql('SELECT translated FROM mesh_ref_copy1', con = db_connection)
# mesh_zh_df.dropna(axis=0, inplace=True)
# print(len(mesh_zh_df))
# mesh_zh_list = mesh_zh_df['translated'].tolist()
# mesh_zh_dict = []
# for item in mesh_zh_list:
#     mesh_zh_dict += item.split('，')
# removed_list = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '', ' ']
# mesh_zh_dict = [w for w in mesh_zh_dict if w not in removed_list]
# mesh_zh_dict = list(set(mesh_zh_dict))

In [18]:
nsfc_data = pd.read_sql('SELECT summaryCh FROM nsfc_data', con = db_connection)
nsfc_data.dropna(axis=0, inplace=True)
print(len(nsfc_data))

71469


In [22]:
for idx, text in nsfc_data.iterrows():
    if "乳腺癌激酶" in text['summaryCh']:
        print(idx+1)

In [14]:
mesh = pd.read_csv('./data/a.csv')
mesh.dropna(axis=0, inplace=True, subset=['translated'])
mesh_zh_dict = mesh['translated'].tolist()
mesh_zh_dict = list(set(mesh_zh_dict))
print(len(mesh_zh_dict))

/home/wuyaqiang/software/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


42886


In [15]:
chinese_word_dict = pickle.load(open('./dict data/chinese_word_dict.pickle', 'rb'))
print(len(chinese_word_dict))
chinese_word_dict += mesh_zh_dict
chinese_word_dict = list(set(chinese_word_dict))
print(len(chinese_word_dict))

54609
91563


In [108]:
jieba.load_userdict('./dict data/chinese_word_dict.txt')
stopwords = [line.strip() for line in codecs.open('./stopwords-zh.txt', 'r', encoding='utf8').readlines()]

In [116]:
# all_question = pd.read_csv('./data/description.csv')
Herceptin_question = pd.read_excel('./data/Herceptin.xlsx')
# all_question = pd.read_excel('./data/Herceptin_funding_filtered.xlsx')
# all_question = pd.read_excel('./data/Herceptin_patient_filtered.xlsx')
Kadcyla_question = pd.read_excel('./data/Kadcyla.xlsx')
Perjeta_1_question = pd.read_excel('./data/Perjeta_1.xlsx')
Perjeta_2_question = pd.read_excel('./data/Perjeta_2.xlsx')
Tecentriq_question = pd.read_excel('./data/Tecentriq.xlsx')
Xeloda_question = pd.read_excel('./data/Xeloda.xlsx')
No_product_question = pd.read_excel('./data/No_product_merged.xlsx')

all_question = pd.concat([Herceptin_question, Kadcyla_question, Perjeta_1_question, Perjeta_2_question, Tecentriq_question, Xeloda_question, No_product_question])
print('all_question: ', len(all_question))
all_question.head(5)

all_question.dropna(axis=0, inplace=True, subset=['Description'])
all_question.isna().sum()

print('all_question: ', len(all_question))

all_question:  1970
all_question:  1656


In [ ]:
import jieba.analyse.textrank as textrank

def keyword_extract(text, word_dict):
#     text = list(jieba.cut(text, cut_all=False))
#     text = list(jieba.cut_for_search(text))
#     text = [w for w in text if w in word_dict]
    print(textrank(text))
    return " ".join(textrank(text))

# 去除所有“[]”内的文本
def remove_special_string(text):
    text = re.sub(r'\[[^]]*\]', '', text)
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'（[^）]*\）', '', text)
    text = re.sub(r'\<[^>]*\>', '', text)
    return text

def remove_stopwords(text, stopwords):
    keywords = text.split()
    keywords = [w for w in keywords if w not in stopwords]
    return "  ".join(keywords)

all_question['Description'] = all_question.Description.apply(lambda x: remove_special_string(x))
all_question['KeyWords'] = all_question.Description.apply(lambda x: keyword_extract(x, chinese_word_dict))
# all_question['KeyWords'] = all_question.KeyWords.apply(lambda x: remove_stopwords(x, stopwords))

In [126]:
keyword_extract("关于拉帕替尼加希罗达的临床试验", chinese_word_dict)

[]


''

In [123]:
for idx, row in all_question.iterrows():
    if "关于拉帕替尼加希罗达的临床试验" in row['Description']:
        print('111'+row['KeyWords'])

111


In [114]:
description_list = all_question['Description'].tolist()
keywords_list = all_question['KeyWords'].tolist()
print(len(description_list))
print(len(keywords_list))
write_list = []
for idx, item in enumerate(description_list):
    line = '原文本： ' + item + '\n' + '关键词： ' + keywords_list[idx] + '\n\n'
    write_list.append(line)

f_sourcetext_and_keywords = open('text_keywords_3.txt', 'w')
f_sourcetext_and_keywords.writelines(write_list)
f_sourcetext_and_keywords.close()

1656
1656


In [17]:
"乳腺癌激酶" in chinese_word_dict

False

In [41]:
case_number_list = []
description_list = []
for idx, row in No_product_question.iterrows():
    if str(row['Case Number']) != 'nan':
        case_number_list.append(row['Case Number'])
        description_list.append(str(row['Description']))
    else:
        description_list[-1] = description_list[-1] + " " + str(row['Description'])
print(len(case_number_list))
print(len(description_list))
print(description_list[-1])

563
563
文献查询 Dear：麻烦帮忙查一下以下的文献1，J Natl Cancer Inst. 2018 Jun 1;110(6):560-567. doi: 10.1093/jnci/djy018 2，JAMA Oncol. 2018 Sep 1;4(9):1207-1213. doi: 10.1001/jamaoncol.2018.1436 3，Ann Surg. 2017 Jul 24. doi: 10.1097/SLA.0000000000002439 4，Ann Surg. 2018 May;267(5):946-951. doi: 10.1097/SLA.0000000000002313 5，JAMA Surg. 2017 Jul 1;152(7):665-670. doi: 10.1001/jamasurg.2017.0562非常感谢


In [ ]:
row_list = list(zip(case_number_list, description_list))
print(len(row_list))
row_list

In [ ]:
No_product_question_processed = pd.DataFrame(row_list, columns=['Case Number', 'Description'])
No_product_question_processed

In [46]:
No_product_question_processed.to_excel("./data/No_product_merged.xlsx")

In [49]:
all_data = pd.read_excel("./data/Herceptin_filtered.xlsx")

In [51]:
all_data.isna().sum()

Case Number                400
Contact Type               402
Contact Channel            402
Response Channel           402
Enquiry Classification     402
Subject                    402
Description                332
Product                    402
Response                    95
Unnamed: 9                1157
dtype: int64

In [52]:
all_data.dropna(axis=0, inplace=True, subset=['Case Number', 'Contact Type'])

In [53]:
all_data.isna().sum()

Case Number                 0
Contact Type                0
Contact Channel             0
Response Channel            0
Enquiry Classification      0
Subject                     0
Description                 0
Product                     0
Response                    0
Unnamed: 9                756
dtype: int64

In [54]:
len(all_data)

756

In [56]:
all_data.drop(all_data[all_data['Contact Type'] == 'Patient'].index, inplace=True)

In [57]:
len(all_data)

201

In [58]:
# all_data.to_excel('./data/Herceptin_patient_filtered.xlsx')

In [2]:
mesh_data = pd.read_csv('./mesh_ref_copy1.csv')

In [7]:
print(len(mesh_data))
mesh_data.head()

911261


,1000065,anatomy category,A
0,1000066,organisms category,B
1,1000067,diseases category,C
2,1000068,chemicals and drugs category,D
3,1000069,"analytical, diagnostic and therapeutic techniq...",E
4,1000070,psychiatry and psychology category,F


In [10]:
def is_zh(string):
    return all('\u4e00' <= char <= '\u9fff' for char in string)

In [16]:
chinese_word_dict = []
for idx,row in mesh_data.iterrows():
    if is_zh(row['anatomy category']):
        chinese_word_dict.append(row['anatomy category'])

In [22]:
print(len(chinese_word_dict))
# chinese_word_dict

44223


In [20]:
mimc_data = pd.read_csv('./mimc_not_matched.csv')

In [25]:
for idx, row in mimc_data.iterrows():
    chinese_word_dict.append(row['zh'])
print(len(chinese_word_dict))

59002


In [29]:
cnkd_data = pd.read_csv('./cnkd_not_matched.csv')
print(len(cnkd_data))

6555


In [31]:
for idx, row in cnkd_data.iterrows():
    chinese_word_dict.append(row['zh'])
print(len(chinese_word_dict))

65557


In [32]:
chinese_word_dict = list(set(chinese_word_dict))

In [33]:
len(chinese_word_dict)

54609

In [38]:
'赫赛汀' in chinese_word_dict

True

In [39]:
import pickle
with open('./chinese_word_dict.pickle', 'wb') as f:
    pickle.dump(chinese_word_dict, f)

In [99]:
f_chinese_word_dict = open('./dict data/chinese_word_dict.txt', 'w')
write_list = [word+"\n" for word in chinese_word_dict]
f_chinese_word_dict.writelines(write_list)
f_chinese_word_dict.close()